# Project 2:
## Authors: Frederick Nilsen, Mia Rødde, Sara Abnar

### Warnings

* Ikke endre uten at de andre vet det
* **Alltid** gå på Kernel -> Restart & Clear Output
* Ikke skriv utenfor cellene og sånn

## Implementation of set 1 - points in 2D

In [ ]:
## IMPORTS

import numpy as np
import matplotlib.pyplot as plt
from loader import *
from plotting import *
from spirals import get_data_spiral_2d
import pickle

In [ ]:
## GLOBAL CONSTS AND DEFINITIONS

sigma = lambda x : np.tanh(x)         # Activation function
eta = lambda x : 0.5*(1+np.tanh(x/2)) # Scalar function     
eta_ddx = lambda x :  0.25*(1-np.tanh(x/2)*np.tanh(x/2)) # Derivative of eta
sigma_ddx = lambda x: 1-(np.tanh(x))**2 # Derivative of sigma


K = 10 # +1 gives number of layers, arbitrary number
d = 2 # Ehh
tau = 0.1 # learning parameter [0.01,0.1]
np.random.seed(0)
Y0, C = get_data_spiral_2d(200) #Default n=200
I = Y0.shape[1]

np.random.seed(0)
W = np.random.randn(K,d,d)
np.random.seed(0)
b = np.random.randn(K,d,1)     #startverdier
np.random.seed(0)
mu = np.random.randn()
np.random.seed(0)
w = np.random.randn(d,1)
h = 0.1

Ulst = [W, b, w, mu]

In [ ]:
## INITIALISATION CELL
def saveU(dataset="spirals"):
    with open('weights-' + dataset + '.txt', 'wb') as f:
        pickle.dump(Ulst,f)

def loadU(dataset="spirals"):
    with open('weights' + dataset + '.txt','r') as f:
        pickle.load(f)

def getColorFromLabel(C):
    colorLst = []
    colorLst.append('r' if value else 'b' for value in C)
    return colorLst
C_colors = getColorFromLabel(C)


#optimert funksjon
def YK(W, b, Ystart=Y0):
    #tom Y-matrise, 3-dimensjonalt numpy-array
    Ym = np.zeros((K+1,d,I))
    Ym[0,:,:] = np.copy(Ystart)
    k=0
    while k < K:
        Ym[k+1] = Ym[k] + h*sigma(W[k]@Ym[k] + b[k])
        k+=1
    return Ym

#Finner P_K
def getGradients(K, W, b, w, mu, c=C, Ystart=Y0):
    Y_all = YK(W,b, Ystart)
    lastY = Y_all[-1,:,:]
    P = np.zeros((Y_all.shape)) # init matrix
    P[K,:,:] = w@np.transpose((Zf(w,mu, lastY)-c)*eta_ddx(np.transpose(lastY)@w+mu))  #(7)
    
    dJdMU = eta_ddx(np.transpose(np.transpose(lastY)@w+mu))@(Zf(w,mu, lastY)-c)   #(5), kunne også brukt '.T' for transpose
    dJdw = lastY@((Zf(w,mu, lastY)-c)*eta_ddx(np.transpose(lastY)@w+mu))              #(6)
    for k in range(K,0,-1):
        P[k-1,:,:] = P[k,:,:]+h*np.transpose(W[k-1,:,:])@(sigma_ddx(W[k-1,:,:]@Y_all[k-1,:,:]+b[k-1,:,:])*P[k,:,:])  #(8) 
    dJdWk = np.zeros((K,d,d))
    dJdBk = np.zeros((K,d,1))
    
    for k in range(0,K):
        dJdWk[k] = h*(P[k+1]*sigma_ddx(W[k,:,:]@Y_all[k,:,:]+b[k,:,:]))@np.transpose(Y_all[k,:,:])  #(9)
        dJdBk[k] = h*(P[k+1]*sigma_ddx(W[k,:,:]@Y_all[k,:,:]+b[k,:,:]))@np.ones((I,1)) #One not needed?#(10)
    return dJdWk, dJdBk, dJdw, dJdMU


#Adam descent algorithm
def AdamAlg(Ulst, m, v, c=C, Ystart=Y0):
    beta1 = 0.9
    beta2 = 0.999
    alpha = 0.01
    epsilon = 1e-8
    W, b, w, mu = Ulst
    gradLst = [x for x in getGradients(K, W, b, w, mu, c, Ystart)]
    for i in range(len(gradLst)):
        g = gradLst[i]
        m[i] = beta1*m[i]+(1-beta1)*g
        v[i] = beta2*v[i]+(1-beta2)*(g*g)
        mhat = m[i]/(1-beta1**(counter))
        vhat = v[i]/(1-beta2**(counter))
        Ulst[i] -= alpha*mhat/(np.sqrt(vhat)+epsilon)
    return Ulst, m, v

In [ ]:
#Lager siste Y_k = Y_K
YK_list = YK(W,b)
lastY = YK_list[-1,:,:]

Zf = lambda w,mu,finalY=lastY : eta(np.transpose(finalY)@w +mu)
Z=Zf(w,mu)
Jf = lambda Z=Zf(w,mu), ce=C : 1/2*np.linalg.norm(Z-ce)**2 #Cost function
J = Jf()

In [ ]:
#Algorithm
converged = False
counter = 1

#initializing list
m = [0 for x in range(len(Ulst))]
v = [0 for x in range(len(Ulst))]

#for AdamAlg
for i in range(1,10001):   #Skal være 40001
    Ulst, m, v = AdamAlg(Ulst, m, v)
    counter +=1 

W, b, w, mu = Ulst
saveU()

In [ ]:
testY, testC = get_data_spiral_2d(200)
trainedY = YK(W, b, testY)
lastY = trainedY[-1,:,:]
print(Jf(Zf(w,mu, lastY), testC))

#forward function
def classifications(K, W, b, w, mu):
    def classifying_unknown_points(Y0):
        Y = YK_list[-1,:,:]
        Z = Zf(w, mu, Y)
        return Z
    return classifying_unknown_points
f = classifications(K, W, b, w, mu)
 
#Last function
def classifications2(w, mu):
    def classifying_unknown_points2(Y0):
        Z = Zf(w, mu, lastY)
        return Z
    return classifying_unknown_points2
g = classifications2(w, mu)

plot_progression(trainedY, testC)
#plot_model(f, lastY, testC, I)
#plot_separation(g, lastY, testC, I)

In [ ]:
lowerBoundary_setTrueZ = [0.65, 0.75, 0.85, 0.95, 0.99]

def checkResults(n=I):
    testY, testC = get_data_spiral_2d(n)
    trainedY = YK(W, b, testY)
    lastY = trainedY[K,:,:]
    trainedZ = Zf(w,mu,lastY)

    J_new=Jf(trainedZ,testC)
    numTrue=0
    numJTrue = 0
    print("J_new:", J_new)
    numTrue = sum(1 if testC[i][0] else 0 for i in range(n))
    print("Antall true:", numTrue)
    print("Lower boundary \t Antall true i følge J \t\t Success rate[%]")
    for lowerBound in lowerBoundary_setTrueZ:
        numTrue_J = sum(1 if trainedZ[i][0] >=lowerBound else 0 for i in range(n))
        success_rate = round(numTrue_J/numTrue*100, 5)
        print(lowerBound, numTrue_J, success_rate, sep="\t\t\t")
    

lastY = trainedY[K,:,:]
trainedZ = Zf(w,mu,lastY)

In [ ]:
checkResults()

In [ ]:
#INITIALISING LIST, PART 2
K = 10 #number of layers, arbitrary number
Y0pic, C_pic = get_dataset("testing")
d = Y0pic.shape[0]
I = Y0pic.shape[1]

np.random.seed(0)
W = np.random.randn(K,d,d)
np.random.seed(0)
b = np.random.randn(K,d,1)     #startverdier
np.random.seed(0)
mu = np.random.randn()
np.random.seed(0)
w = np.random.randn(d,1)
h = 0.1

Ulst = [W, b, w, mu]

In [ ]:
#algorithm for pictures
counter = 1

#initializing list
m = [0 for x in range(len(Ulst))]
v = [0 for x in range(len(Ulst))]

#for AdamAlg
for i in range(1,101):   #Skal være 40001
    Ulst, m, v = AdamAlg(Ulst, m, v, C_pic, Y0pic)
    print(counter)
    counter +=1

saveU("MNIST")

In [ ]:
W, b, w, mu = Ulst
testpic, testCp = get_dataset("testing")
I = testpic.shape[1]
d = testpic.shape[0]
trainedPic = YK(W, b, testpic)
lastYPic = trainedPic[K,:,:]

trainedZ = Zf(w, mu, lastYPic)

#koden går mye raskere men J er jo alt for høy: 117.54204154098629

In [ ]:
lowerBoundary_setTrueZ = [0.65, 0.75, 0.85, 0.95, 0.99, 1.00]

def checkResultsPic(n=I): 
    J_new=Jf(trainedZ,testCp)
    numTrue_J = np.array([]) 
    print("J_new:", J_new)
    numTrue = sum(1 if testCp[i][0] else 0 for i in range(n))
    print("Antall true:", numTrue)
    print("Lower boundary \t Antall true i følge J \t\t Success rate[%]")
    for lowerBound in lowerBoundary_setTrueZ:
        numTrue_J = sum(1 if trainedZ[i][0] >=lowerBound else 0 for i in range(n))
        success_rate = round(numTrue_J/numTrue*100, 5)
        print(lowerBound, numTrue_J, success_rate, sep="\t\t\t")

checkResultsPic()